<a href="https://colab.research.google.com/github/ucheokechukwu/ml_tensorflow_deeplearning/blob/main/07_milestone_project_1_food_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check GPU

Mixed precision training - uses both 16 and 32 bit types which makes it run faster during training.
Special hardware units called tensor cores.
Google colab offers:
K80 (not compatible)
P100 (not compatible)
Tesla T4 compatible


## Getting the helper functions
In past modules, we have helper functions that we import instead of re-writing the script.

In [1]:
# download helper function script

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-03-05 20:46:04--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-03-05 20:46:04 (103 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
# import series of helper functions for the notebook 
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

## Use Tensorflow Datasets to download data
Tensorflow Datasets is a collection of ready-to-use datasets (already in Tensorflow format) with Tensorflow  or other Python ML frameworks. All datasets are exposed as `tf.data.Datasets`, enabling easy to use and high-performanc input pipelines.

https://www.tensorflow.org/datasets

Why use this?
- load data already in tensor format
- practice in well established datasets
- experiment with different modelling techniques on a consistent dataset

why not use this?
- datasets are static and do not change like real-world datasets

In [3]:
# Get tensorflow dataset
import tensorflow_datasets as tfds

In [4]:
# list all available datasets
datasets_list = tfds.list_builders() # get all available datasets in tfds
print("food101" in datasets_list) # is our target in the list of tfds

True


In [5]:
!nvidia-smi

Sun Mar  5 20:46:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    51W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

* note that this is a lot and takes a long time

In [ ]:
(train_data, test_data), ds_info = tfds.load(name="food101",
                                             split=["train","validation"],
                                             shuffle_files=True,
                                             as_supervised=True, #data gets returned in tuple format (data,label)
                                             with_info=True) #metadata is downloaded as well and saved into ds_info

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

## Exploring the Food1010 data from Tensorflow datasets

To become one with our data, we need to find:
* Class names
* The shape of our input data (image tensors)
* The datatype of our input data
* What the labels looks like (e.g. are they one-hot encoded or are they label encoded?)
* Do the labels match up with the class names?


In [ ]:
# features of food101 from TFDS
ds_info.features

In [ ]:
# get the class names (we have to dig into the metadata ds_info)
class_names = ds_info.features["label"].names
class_names[:10]

In [ ]:
# get one sample of our training data (remember: start small and increase complexity)
train_data

In [ ]:
train_one_sample = train_data.take(10)
train_one_sample

In [ ]:
# output info about our training sample
for image, label in train_one_sample:
  print(f"""
  image_shape: {image.shape}
  image_datatype: {image.dtype}
  target class from food101 (tensor form): {label}
  class name (str form): {class_names[label.numpy()]}
  """)

In [ ]:
import tensorflow as tf

In [ ]:
# what does our image tensor look like?
image

In [ ]:
# what are the min and max values of our image tensor?
tf.reduce_min(image), tf.reduce_max(image)

## Let's plot an image from our dataset

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image)
plt.title(class_names[label.numpy()]) # add title to verify the label is associated with the right image
plt.axis(False)

## Create preprocessing functions for our data
Neural networks perform best when data is in a certain way (e.g. batched, normalized, etc).
However not all data comes like this.
In order to get it ready for a neural network, we have to write preprocessing functions and map it into your data.

What we know about our data
1. it's in uint8 datatype
2. it's comprised of all different sizes of image tensors
3. it's not scaled i.e. pixel values are 0-255 not 0-1.

what we know that models like:
1. data in `float32` dtype (or for mixed precision, `float16` and `float32`)
2. for batches, Tensorflow likes all the tensors within a batch to be all of the same sizes
3. scaled/normalized tensors always perform better

Therefore we have a framework for preprocessing function.

Since we're going to be using an `EfficientNetBX` pretrained model from `tf.keras.applications`, we don't need to rescale our data (these architectures have rescaling built in). 

This means our function needs to: 
1. reshape our images to all the same size
2. convert the dtype of our image tensors from uint8 to `float 32`

In [ ]:
# make a functio for preprocessing images
def preprocess_img(image, label, img_shape=224):
  """
  converts image dataype from 'unint8' -> 'float32' and
  reshapes image to [img_shape, img_shape, colour_channels]
  """
  image = tf.image.resize(image, [img_shape, img_shape])
  # image = image/255 # resizing function but not required for EfficientNetX
  return tf.cast(image, tf.float32), label # return a tuple of float32 image and a label


In [ ]:
# preprocess single sample miage and check the outputs
preprocessed_img = preprocess_img(image, label)[0]
print(f"image before preprocessing: \n {image[:2]}...., \nShape: {image.shape}, \nDatatype: {image.dtype}")
print(f"image after preprocessing: \n {preprocessed_img[:2]}...., \nShape: {preprocessed_img.shape}, \nDataype: {preprocessed_img.dtype}")

## batch and prepare datasets
https://www.tensorflow.org/guide/data_performance (read the Best Practice Summary)






In [ ]:
# map preprocessing function to training data and pararellize it
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
# as this function runs, autotune means run this in parallel with all the available CPU
# shuffle train_data and turn it into batches and prefetch it (load it faster)
train_data = train_data.batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE) #.shuffle(buffer_size=1000)
# how many elements to shuffle at a single time, determined by the amount of memory
# note that shuffling is redundant, we already did it before but we're repeating it here to be sure

test_data = test_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size=32).prefetch(tf.data.AUTOTUNE).cache()

In [ ]:
train_data, test_data

> "Hey, Tensorflow, map this preprocessing function (`preprocess_img`) across our training dataset, then `shuffle` a number of elements and `batch` them together, and finally make sure you prepare new batches (`prefetch`) whilst the model is looking through (finding patterns) in the current batch"


## creating modelling callbacks
- callbacks are toold which add helpful functionality to models during training
1. Tensorboard callback
2. ModelCheckpoint callback

In [ ]:
# creating tensorboard callback
from helper_functions import create_tensorboard_callback

In [ ]:
# creating modelcheckpoint callback
checkpoint_path = "model_checkpoints/cp.ckpt"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      monitor="val_acc",
                                                      save_weights_only=True,
                                                      verbose=0,
                                                      save_best_only=True
)

## setup mixed precision training



In [ ]:
# turn on mixed precision training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16") # set global policy to mixed precision

In [ ]:
!nvidia-smi

In [ ]:
mixed_precision.global_policy()

* Calculations are done in float16 and variables are stored in float32. So the outputs dtype has to be changed to float32 for numeric stability

## build feature extraction model

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
# create base model
input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# create functional model
inputs = layers.Input(shape=input_shape, name="input_layer")

x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(len(class_names))(x)
outputs = layers.Activation("softmax", dtype = tf.float32, name = "softmax_float32")(x)
model = tf.keras.Model(inputs, outputs)

# note that we separated the activation layer from the dense layer so that we can set the output to dtype float32

In [ ]:
# compile the model
# note that the label is in integer form not one hot encoded

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics="accuracy")
model.summary()

## Checking layer dtype policy... are we using mixed precision

In [ ]:
# Check layer dtype policy

for layer in model.layers:
  print (layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

Going through the above we see:
* `layer.name`: the human readable name of a particular layer
* `layer.trainable`: is the layer trainable or not?
* `layer.dtype`: is the datatype teh layer store stores variables in
* `layer.dtype_policy`: the data type policy a layer computes on its variables with

In [ ]:
for layer in base_model.layers:
  print (layer.name, layer.trainable, layer.dtype, layer.dtype_policy)


In [ ]:
history = model.fit(train_data,
                    epochs=3,
                    steps_per_epoch = len(train_data),
                    validation_data=test_data,
                    validation_steps=0.15*len(test_data),
                    callbacks=[create_tensorboard_callback(dir_name="training_logs", experiment_name="efficientnetb0_101_all_data"), model_checkpoint
                                                 ])

In [ ]:
results=model.evaluate(test_data)

## Challenge: 
Complete the template version of 07 to obtain a computer vision model building off the one that was built in this notebook to beat the DeepFood papaer
